In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jul  1 06:25:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade mxnet-cu110
!pip install --upgrade gluoncv
!pip install ensemble_boxes

     |████████████████████████████████| 323.5MB 53kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.3MB 11.9MB/s 


In [ ]:
import shutil
import pandas as pd
import numpy as np 
import os
import cv2
import re

from PIL import Image

from matplotlib import pyplot as plt
from datetime import datetime

from sklearn.model_selection import StratifiedKFold, train_test_split
import joblib
import sys
#sys.path.insert(0, "/kaggle/input/yolo-v5-by-ultralytics/yolov5-master/")
#sys.path.insert(0, "../input/weightedboxesfusion")

In [ ]:
if(os.path.isdir('test')):
  print('folder exists, skip unzip procedure')
else:
  shutil.copyfile('/content/drive/MyDrive/aicrowd_wheat/test.zip','/content/test.zip')
  shutil.unpack_archive('./test.zip')
shutil.copyfile('/content/drive/MyDrive/aicrowd_wheat/submission.csv','/content/submission.csv')

'/content/submission.csv'

In [ ]:
len(os.listdir('./test/')), os.listdir('./test/')[0:3]

(2856,
 ['d2a39bd972b70be9337c1637c76626d954cd9927e1aedff42df5dccd4ed1cab6.png',
  'b05f6efc3a63c75eb7760ec203a8c6abe7391745172a862f036671bf11951d73.png',
  'c6d0bbd6141bd93c59c2925d0bd83559aaa58cf929e1e34b5cfdcef3d7e64f21.png'])

In [ ]:
test_df = pd.read_csv('submission.csv')
test_df['image_id'] = test_df['image_name']
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2849 entries, 0 to 2848
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  2849 non-null   object
 1   domain      2849 non-null   int64 
 2   PredString  2849 non-null   object
 3   image_id    2849 non-null   object
dtypes: int64(1), object(3)
memory usage: 89.2+ KB


In [ ]:
test_df.head()

,image_name,domain,PredString,image_id
0,c087c8b89904edee56997a25d64f4cd1b7c9441e64889a...,18,no_box,c087c8b89904edee56997a25d64f4cd1b7c9441e64889a...
1,b11a617a64ca6120021e70b107933ddb70bf052395759d...,18,no_box,b11a617a64ca6120021e70b107933ddb70bf052395759d...
2,4fd7a4dc7dc60b5daedc7d57fdf8000ef3cab267654b56...,18,no_box,4fd7a4dc7dc60b5daedc7d57fdf8000ef3cab267654b56...
3,3f60fada17caf269a8d70798eb3bb5771f2c10d13c887e...,18,no_box,3f60fada17caf269a8d70798eb3bb5771f2c10d13c887e...
4,8cdcb28a926c217d0b587fcba2050e283b3175824a5b6e...,18,no_box,8cdcb28a926c217d0b587fcba2050e283b3175824a5b6e...


In [ ]:
import os
import time
import random
import mxnet as mx
import pandas as pd
from mxnet.contrib import amp
import mxnet as mx
import gluoncv as gcv

amp.init()

rounds = 3
max_epochs = 5
learning_rate = 0.001
batch_size = 8
img_s = 1024
threshold = 0.1
context = mx.gpu()

def load_model(path, ctx=mx.cpu()):
    net = gcv.model_zoo.yolo3_darknet53_custom(["wheat"], pretrained_base=False)
    net.set_nms(post_nms=150)
    net.load_parameters(path, ctx=ctx)
    return net

print("Loading test images...")
test_images = [
    (os.path.join(dirname, filename), os.path.splitext(filename)[0])
        for dirname, _, filenames in os.walk('/content/test') for filename in filenames
]
print(len(test_images))

print("Loading trained model...")
model = load_model("/content/drive/MyDrive/aicrowd_wheat/gluon_yolo/global-wheat-yolo3-darknet53_best.params", ctx=context)
# model = load_model("/content/drive/MyDrive/aicrowd_wheat/gluon_yolo/global-wheat-yolo3-darknet53_pseudo.params", ctx=context)

#print("Loading training set...")
#dataset = load_dataset("/content/test/")

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.9.0+cu102` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


Loading test images...
2856
Loading trained model...


In [ ]:
import random
import numpy as np
import gluoncv as gcv
from ensemble_boxes import *

def load_image(path):
    with open(path, "rb") as f:
        buf = f.read()
    return mx.image.imdecode(buf)


def inference(models, path):
    raw = load_image(path)
    rh, rw, _ = raw.shape
    classes_list = []
    scores_list = []
    bboxes_list = []
    for _ in range(5):
        img, flips = gcv.data.transforms.image.random_flip(raw, px=0.5, py=0.5)
        x, _ = gcv.data.transforms.presets.yolo.transform_test(img, short=img_s)
        _, _, xh, xw = x.shape
        rot = random.randint(0, 3)
        if rot > 0:
            x = np.rot90(x.asnumpy(), k=rot, axes=(2, 3))
        for model in models:
            classes, scores, bboxes = model(mx.nd.array(x, ctx=context))
            if rot > 0:
                if rot == 1:
                    raw_bboxes = bboxes.copy()
                    bboxes[0, :, [0, 2]] = xh - raw_bboxes[0, :, [1, 3]]
                    bboxes[0, :, [1, 3]] = raw_bboxes[0, :, [2, 0]]
                elif rot == 2:
                    bboxes[0, :, [0, 1, 2, 3]] = mx.nd.array([[xw], [xh], [xw], [xh]], ctx=context) - bboxes[0, :, [2, 3, 0, 1]]
                elif rot == 3:
                    raw_bboxes = bboxes.copy()
                    bboxes[0, :, [0, 2]] = raw_bboxes[0, :, [1, 3]]
                    bboxes[0, :, [1, 3]] = xw - raw_bboxes[0, :, [2, 0]]
                raw_bboxes = bboxes.copy()
                bboxes[0, :, 0] = raw_bboxes[0, :, [0, 2]].min(axis=0)
                bboxes[0, :, 1] = raw_bboxes[0, :, [1, 3]].min(axis=0)
                bboxes[0, :, 2] = raw_bboxes[0, :, [0, 2]].max(axis=0)
                bboxes[0, :, 3] = raw_bboxes[0, :, [1, 3]].max(axis=0)
            bboxes[0, :, :] = gcv.data.transforms.bbox.flip(bboxes[0, :, :], (xw, xh), flip_x=flips[0], flip_y=flips[1])
            bboxes[0, :, 0::2] = (bboxes[0, :, 0::2] / (xw - 1)).clip(0.0, 1.0)
            bboxes[0, :, 1::2] = (bboxes[0, :, 1::2] / (xh - 1)).clip(0.0, 1.0)
            classes_list.append([
                int(classes[0, i].asscalar()) for i in range(classes.shape[1])
                    if classes[0, i].asscalar() >= 0.0

            ])
            scores_list.append([
                scores[0, i].asscalar() for i in range(classes.shape[1])
                    if classes[0, i].asscalar() >= 0.0

            ])
            bboxes_list.append([
                bboxes[0, i].asnumpy().tolist() for i in range(classes.shape[1])
                    if classes[0, i].asscalar() >= 0.0
            ])
    bboxes, scores, classes = weighted_boxes_fusion(bboxes_list, scores_list, classes_list)
    bboxes[:, 0::2] *= rw - 1
    bboxes[:, 1::2] *= rh - 1
    return bboxes, scores, classes

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    #for j in zip(scores, boxes):
    #    pred_strings.append("{1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    if len(boxes) >0:
      boxes = [" ".join([str(int(i)) for i in item]) for item in boxes]
      BoxesString = ";".join(boxes)
    else:
      BoxesString = "no_box"
    # return ";".join(pred_strings)
    return BoxesString

In [ ]:
df_binary = pd.read_csv('/content/drive/MyDrive/aicrowd_wheat/binary_preds.csv')
df_binary['image_name'] = df_binary['file_name'].str.replace('.png','')
df_binary['prediction'].value_counts()

yes    2785
no       71
Name: prediction, dtype: int64

In [ ]:
print("Inference...")
results = []
threshold = 0.1
for path, image_id in test_images:
    print(path)
    bboxes, scores, classes = inference([model], path)
    # bboxes[:, 2:4] -= bboxes[:, 0:2]
    '''results.append({
        "image_id": image_id,
        "PredictionString": ";".join([
            " ".join([str(x) for x in [round(x) for x in bboxes[i].tolist()]])
                for i in range(classes.shape[0])
                    if model.classes[int(classes[i])] == "wheat" and scores[i] > threshold
        ])
    })'''
    #if(df_binary[df_binary['image_name']==image_id]['prediction'].values[0]=='no'):
    #  threshold = 0.0
    #else:
    #  threshold = 0.6
    boxes = bboxes[scores >= threshold].astype(np.int32)
    scores = scores[scores >= threshold]
    result = {
            'image_id': image_id,
            'predictions': format_prediction_string(boxes, scores),
            # 'probabilities': format_scores_string(boxes, scores)
        }
    results.append(result)
pd.DataFrame(results, columns=['image_id', 'PredictionString']).to_csv('submission.csv', index=False)

In [ ]:
df_results = pd.DataFrame(results, columns=['image_id', 'predictions'])
df_results.head()

,image_id,predictions
0,d2a39bd972b70be9337c1637c76626d954cd9927e1aedf...,588 863 643 1005;352 846 423 993;780 743 841 8...
1,b05f6efc3a63c75eb7760ec203a8c6abe7391745172a86...,285 597 462 729;944 623 1023 764;503 628 676 7...
2,c6d0bbd6141bd93c59c2925d0bd83559aaa58cf929e1e3...,623 715 767 862;299 889 484 943;186 701 294 86...
3,41de21f235d6080e0c7ccd5424a472344d431ac7a4b43f...,83 209 160 341;195 213 261 319;776 49 879 119;...
4,4ca4007eb7f785130aa160375a2eeece75a3b6e1e633ed...,749 966 869 1022;488 136 579 203;277 34 351 14...


In [ ]:
df_submission = test_df[['image_name',
                         'domain']].merge(df_results, 
                                          left_on=['image_name'], right_on=['image_id'])
df_submission = df_submission[['image_name','domain','predictions']]
df_submission.columns = ['image_name','domain','PredString']
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  2849 non-null   object
 1   domain      2849 non-null   int64 
 2   PredString  2849 non-null   object
dtypes: int64(1), object(2)
memory usage: 89.0+ KB


In [ ]:
API_KEY = "9967b67c796b4d50e5d26f4dfd97ed06"
df_submission.to_csv("submission_final.csv", index=False)

In [ ]:
df_submission.to_csv("/content/drive/MyDrive/aicrowd_wheat/gluon_yolo/submission_final.csv", index=False)

In [ ]:
!pip install aicrowd-cli
!aicrowd login --api-key $API_KEY
!aicrowd submission create -c global-wheat-challenge-2021 -f submission_final.csv

     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 174kB 19.3MB/s 
     |████████████████████████████████| 215kB 25.9MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


API Key valid
Saved API Key successfully!
submission_final.csv ━━━━━━━━━━━━━━━━━━ 100.0% • 1.7/1.7 MB • 2.6 MB/s • 0:00:00
                                             ╭─────────────────────────╮                                             
                                             │ Successfully submitted! │                                             
                                             ╰─────────────────────────╯                                             
                                                   Important links                                                   
┌──────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/global-wheat-challenge-2021/submissions/149183              │
│                  │                                                                                                │
│  All submissions │ https://www.aicrowd.com/challe